In [62]:
import pandas as pd
import numpy as np
import rapidfuzz

In [63]:
state='SC'
path = r"C:\Users\david\OneDrive\Documents\GitHub\SPEOC-pt-1\Data\Post1790\SC\Post_1790_South_Carolina_CD.xlsx"
occs_and_debt = pd.read_excel(path,
                             header=11, usecols='G, H, M, N, AB, AC, AQ, AR') #occupations plus continental debt for conneticut

In [64]:
occs_and_debt = occs_and_debt.dropna(subset=['Occupation'])
print(occs_and_debt)

    State  Occupation  Dollars  Cents  Dollars.1  Cents.1  Dollars.2  Cents.2
0      NY    Merchant    690.0   96.0      345.0     48.0      665.0     28.0
1      SC     Mariner     83.0   33.0       41.0     66.0       80.0     43.0
2      SC    Merchant   1017.0   76.0      508.0     88.0      994.0     13.0
3      SC    Merchant     33.0   34.0       16.0     66.0       30.0     65.0
4      SC    Merchant    386.0   32.0      193.0     16.0      222.0     19.0
..    ...         ...      ...    ...        ...      ...        ...      ...
422    RI    Merchant     33.0   33.0       17.0     39.0       31.0     94.0
423    NY    Merchant    120.0   25.0       60.0     12.0       97.0     70.0
424    MD  Shopkeeper     33.0   33.0       16.0     67.0       31.0     67.0
425    MD    Merchant    124.0   33.0       62.0     17.0      118.0     41.0
427    SC    Marriner    180.0   47.0       90.0     24.0      207.0     93.0

[312 rows x 8 columns]


In [65]:
#exclude non states 
occs_and_debt = occs_and_debt.rename(columns={'Unnamed: 11':'State'})
occs_and_debt = occs_and_debt[occs_and_debt['State'].str.contains(state) == True].drop('State', axis=1)
print(occs_and_debt)

    Occupation  Dollars  Cents  Dollars.1  Cents.1  Dollars.2  Cents.2
1      Mariner     83.0   33.0       41.0     66.0       80.0     43.0
2     Merchant   1017.0   76.0      508.0     88.0      994.0     13.0
3     Merchant     33.0   34.0       16.0     66.0       30.0     65.0
4     Merchant    386.0   32.0      193.0     16.0      222.0     19.0
5    Physician    322.0   70.0      161.0     36.0      246.0     69.0
..         ...      ...    ...        ...      ...        ...      ...
398   Merchant    215.0   21.0      107.0     61.0      207.0     47.0
399   Merchant    290.0   99.0      145.0     49.0      279.0      6.0
400   Merchant  10508.0   90.0        NaN      NaN        NaN      NaN
402    Planter    104.0   78.0       52.0     39.0      113.0     96.0
427   Marriner    180.0   47.0       90.0     24.0      207.0     93.0

[264 rows x 7 columns]


In [66]:
occurences_dict = occs_and_debt['Occupation'].value_counts().to_dict()
print(occurences_dict)

{'Merchant': 134, 'Planter': 30, 'Widow': 12, 'Broker': 11, 'Physician': 10, 'Attorney at Law': 8, 'Carpenter': 5, 'Shopkeeper': 4, 'Factor': 3, 'Merchants': 3, 'Executor of Benjamin Tucker': 2, 'Gentleman': 2, 'Sheriff': 2, 'Mariner': 2, 'Blockmaker': 2, 'as treasurer of the State of south carolina': 2, 'Clerk of the Senate': 2, 'Bricklayer': 1, 'Penniston': 1, 'as Guardian to Mary Deborah L. Gowdey': 1, 'Shipwright': 1, 'Executor of Sarah Lampriere': 1, 'Executor of Samuel Bonneau': 1, 'Cabinet Maker': 1, 'Autioneer': 1, 'Administrator to John Harleston Jun': 1, 'Executor of James Peart': 1, 'Printer': 1, 'Assignee of James Simons': 1, 'Executor of William Gibbs': 1, 'Trustee for the Cincinnati of South Carolina': 1, 'Executor Philip Hawkins': 1, 'Vendue Master': 1, 'Executor of William Parker': 1, 'Hair Dresser': 1, 'Executor of Jacob Smiser': 1, 'Executrix of William Savage': 1, 'Comtroller of the Customs': 1, 'Tanner': 1, 'Blacks smith': 1, 'One of Judges of the Courts': 1, 'Plant

In [67]:
totals = occs_and_debt.groupby('Occupation')[['Dollars', 'Dollars.1', 'Dollars.2', 'Cents', 'Cents.1', 'Cents.2']].apply(sum)
totals = totals.reset_index()
print(totals.to_string())

                                      Occupation  Dollars  Dollars.1  Dollars.2        Cents      Cents.1  Cents.2
0            Administrator to John Harleston Jun     33.0       16.0       30.0    33.000000    66.000000     68.0
1                 Administrator to Joseph Warley   1671.0      835.0      601.0    44.000000    72.000000     64.0
2                       Assignee of James Simons    345.0      172.0      321.0     2.000000    51.000000     84.0
3                                Attorney at Law   3688.0     1842.0     2286.0   294.000000   351.000000    365.0
4                                      Autioneer     10.0        5.0        0.0    65.000000    32.000000      0.0
5                                   Blacks smith   1423.0      711.0      365.0    48.000000    74.000000     55.0
6                                     Blockmaker   5263.0     2631.0     2697.0   123.000000   111.000000     60.0
7                                     Bricklayer     37.0       18.0       18.0 

In [68]:
occs_debt_wo_dups = totals.copy()
#use fuzzy string matching to remove any similar names like merchant and merchents. does not take into account longer titles.
occ_list = occs_debt_wo_dups['Occupation'].tolist()

for occ in occ_list:
    for occ_2 in occ_list:
        if occ != occ_2:
            if rapidfuzz.fuzz.ratio(occ, occ_2) >= 80 and occ != occ_2:
                #transfer amount in row that will be deleted to other row
                row_w_occ_2 = occs_debt_wo_dups.loc[occs_debt_wo_dups['Occupation'] == occ_2] #locate row that contains occupation soon to be deleted
                dollars_in_occ_2 = row_w_occ_2[['Dollars', 'Dollars.1', 'Dollars.2']].sum(axis=1).values #get total dollars
                cents_in_occ_2 = row_w_occ_2[['Cents', 'Cents.1', 'Cents.2']].sum(axis=1).values #get total cents
                dollars_in_occ_2 = dollars_in_occ_2[0] #stored in array. need to retrieve value
                cents_in_occ_2 = cents_in_occ_2[0] #stored in array. need to retrieve value  
                occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ), 'Dollars'] += dollars_in_occ_2 #add dollars to occupation 1
                occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ), 'Cents'] += cents_in_occ_2 #add cents to occupation 1
                 
                #remove value from occurences dictionary and add one to other occupation
                occurences_dict[occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ), 'Occupation'].values[0]] += occurences_dict[occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ_2), 'Occupation'].values[0]] 
                occurences_dict.pop(occ_2)
                
                #remove row
                occs_debt_wo_dups.drop(occs_debt_wo_dups[occs_debt_wo_dups['Occupation'] == occ_2].index, inplace=True) 
                occ_list.remove(occ_2)

occs_debt_wo_dups = occs_debt_wo_dups.reset_index(drop=True) #reset indexes 0-65 

#take the first two words of every job if applicable and remove any duplicate titles. handles longer titles 
two_word_occs = {}
#retrieve all two worded occupations
for occ in occ_list:
    if len(occ.split()) >= 2: #only account for words that are actually more than two words long
        occ_two_word = occ.split()[0] + " " + occ.split()[1] #get first two words
        two_word_occs[occ_list.index(occ)] = occ_two_word #append both words to dict

#retrieve any duplicates now 
flipped = {} 
for key, value in two_word_occs.items():
    flipped.setdefault(value, list()).append(key) #flip keys with values

keys_w_dups = [key for key, values in flipped.items() if len(values) > 1] #retrieve keys that have multiple values. store them in a list

print(occurences_dict)

{'Merchant': 137, 'Planter': 30, 'Widow': 12, 'Broker': 11, 'Physician': 10, 'Attorney at Law': 8, 'Carpenter': 5, 'Shopkeeper': 4, 'Factor': 3, 'Executor of Benjamin Tucker': 2, 'Gentleman': 2, 'Sheriff': 2, 'Mariner': 3, 'Blockmaker': 2, 'as treasurer of the State of south carolina': 2, 'Clerk of the Senate': 2, 'Bricklayer': 1, 'Penniston': 1, 'as Guardian to Mary Deborah L. Gowdey': 1, 'Shipwright': 1, 'Executor of Sarah Lampriere': 1, 'Executor of Samuel Bonneau': 1, 'Cabinet Maker': 1, 'Autioneer': 1, 'Administrator to John Harleston Jun': 1, 'Executor of James Peart': 1, 'Printer': 1, 'Assignee of James Simons': 1, 'Executor of William Gibbs': 1, 'Trustee for the Cincinnati of South Carolina': 1, 'Executor Philip Hawkins': 1, 'Vendue Master': 1, 'Executor of William Parker': 1, 'Hair Dresser': 1, 'Executor of Jacob Smiser': 1, 'Executrix of William Savage': 1, 'Comtroller of the Customs': 1, 'Tanner': 1, 'Blacks smith': 1, 'One of Judges of the Courts': 1, 'Planter as Executor E

In [69]:
print(occs_debt_wo_dups.to_string())

                                      Occupation  Dollars  Dollars.1  Dollars.2        Cents      Cents.1  Cents.2
0            Administrator to John Harleston Jun     33.0       16.0       30.0    33.000000    66.000000     68.0
1                 Administrator to Joseph Warley   1671.0      835.0      601.0    44.000000    72.000000     64.0
2                       Assignee of James Simons    345.0      172.0      321.0     2.000000    51.000000     84.0
3                                Attorney at Law   3688.0     1842.0     2286.0   294.000000   351.000000    365.0
4                                      Autioneer     10.0        5.0        0.0    65.000000    32.000000      0.0
5                                   Blacks smith   1423.0      711.0      365.0    48.000000    74.000000     55.0
6                                     Blockmaker   5263.0     2631.0     2697.0   123.000000   111.000000     60.0
7                                     Bricklayer     37.0       18.0       18.0 

In [70]:
print(flipped)
print("\n\n")
print(keys_w_dups)

{'Administrator to': [0, 1], 'Assignee of': [2], 'Attorney at': [3], 'Blacks smith': [5], 'Cabinet Maker': [9], 'Clerk of': [11], 'Comtroller of': [12], 'Executor Philip': [13], 'Executor of': [14, 15, 16, 17, 18, 19, 20], 'Executor to': [21], 'Executrix of': [22], 'Hair Dresser': [25], 'Merchant as': [28], 'One of': [29], 'Planter as': [33], 'Trustee for': [40], 'Vendue Master': [41], 'as Guardian': [43], 'as treasurer': [44]}



['Administrator to', 'Executor of']


In [71]:
print(occurences_dict)

for occ in keys_w_dups:
    i = 0
    while i < len(flipped[occ]):
        index = flipped[occ][i]
        if len(flipped[occ]) > 1: #make sure to keep one index in the list. don't want to delete ALL occurences
            duplicate_row = occs_debt_wo_dups.loc[[index]] #return row that has index
            dollars_in_dup_row = duplicate_row[['Dollars', 'Dollars.1', 'Dollars.2']].sum(axis=1).values #sum all dollars in row                
            cents_in_dup_row = duplicate_row[['Cents', 'Cents.1', 'Cents.2']].sum(axis=1).values #return total cents in row
            dollars_dup_row = dollars_in_dup_row[0] #stored in array. need to retrieve value
            cents_dup_row = cents_in_dup_row[0] #stored in array. need to retrieve value

            last_index = flipped[occ][len(flipped[occ]) - 1] #get last value in list. we will add dollars and cents here
                        
            occs_debt_wo_dups.loc[last_index, 'Dollars'] += dollars_dup_row #add dollars from one occupation to the other
            occs_debt_wo_dups.loc[last_index, 'Cents'] += cents_dup_row #add cents from one occupation to the other
            
            #remove value from occurences dictionary and add one to other occupation
            
            #print("Removed: " + str(occs_debt_wo_dups.loc[index, 'Occupation']))
            #print('Kept: ' + str(occs_debt_wo_dups.loc[last_index, 'Occupation']))
            
            print(occs_debt_wo_dups.loc[last_index, 'Occupation'])
            occurences_dict[occs_debt_wo_dups.loc[last_index, 'Occupation']] += occurences_dict[occs_debt_wo_dups.loc[index, 'Occupation']] 
            occurences_dict.pop(occs_debt_wo_dups.loc[index, 'Occupation'])
            
            print("Length: " + str(len(flipped[occ])))
            print("Last Index: " + str(last_index) + " Index: " + str(index))
            print(flipped[occ])
            print("\n\n")
                        
            occs_debt_wo_dups.drop(index, inplace=True) #remove duplicate occupation now
            flipped[occ].remove(index) #remove from list
            
            #print(occurences_dict)
            #print("\n\n")
        else:
            i += 1

occs_debt_wo_dups = occs_debt_wo_dups.reset_index(drop=True) #reset indexes 0-...

{'Merchant': 137, 'Planter': 30, 'Widow': 12, 'Broker': 11, 'Physician': 10, 'Attorney at Law': 8, 'Carpenter': 5, 'Shopkeeper': 4, 'Factor': 3, 'Executor of Benjamin Tucker': 2, 'Gentleman': 2, 'Sheriff': 2, 'Mariner': 3, 'Blockmaker': 2, 'as treasurer of the State of south carolina': 2, 'Clerk of the Senate': 2, 'Bricklayer': 1, 'Penniston': 1, 'as Guardian to Mary Deborah L. Gowdey': 1, 'Shipwright': 1, 'Executor of Sarah Lampriere': 1, 'Executor of Samuel Bonneau': 1, 'Cabinet Maker': 1, 'Autioneer': 1, 'Administrator to John Harleston Jun': 1, 'Executor of James Peart': 1, 'Printer': 1, 'Assignee of James Simons': 1, 'Executor of William Gibbs': 1, 'Trustee for the Cincinnati of South Carolina': 1, 'Executor Philip Hawkins': 1, 'Vendue Master': 1, 'Executor of William Parker': 1, 'Hair Dresser': 1, 'Executor of Jacob Smiser': 1, 'Executrix of William Savage': 1, 'Comtroller of the Customs': 1, 'Tanner': 1, 'Blacks smith': 1, 'One of Judges of the Courts': 1, 'Planter as Executor E

In [72]:
print(occs_debt_wo_dups.to_string())

                                      Occupation  Dollars  Dollars.1  Dollars.2        Cents      Cents.1  Cents.2
0                 Administrator to Joseph Warley   1750.0      835.0      601.0   211.000000    72.000000     64.0
1                       Assignee of James Simons    345.0      172.0      321.0     2.000000    51.000000     84.0
2                                Attorney at Law   3688.0     1842.0     2286.0   294.000000   351.000000    365.0
3                                      Autioneer     10.0        5.0        0.0    65.000000    32.000000      0.0
4                                   Blacks smith   1423.0      711.0      365.0    48.000000    74.000000     55.0
5                                     Blockmaker   5263.0     2631.0     2697.0   123.000000   111.000000     60.0
6                                     Bricklayer     37.0       18.0       18.0    35.000000    68.000000     75.0
7                                         Broker   2330.0     1162.0      929.0 

In [73]:
print(flipped)

{'Administrator to': [1], 'Assignee of': [2], 'Attorney at': [3], 'Blacks smith': [5], 'Cabinet Maker': [9], 'Clerk of': [11], 'Comtroller of': [12], 'Executor Philip': [13], 'Executor of': [20], 'Executor to': [21], 'Executrix of': [22], 'Hair Dresser': [25], 'Merchant as': [28], 'One of': [29], 'Planter as': [33], 'Trustee for': [40], 'Vendue Master': [41], 'as Guardian': [43], 'as treasurer': [44]}


In [74]:
print(occurences_dict)

{'Merchant': 137, 'Planter': 30, 'Widow': 12, 'Broker': 11, 'Physician': 10, 'Attorney at Law': 8, 'Carpenter': 5, 'Shopkeeper': 4, 'Factor': 3, 'Gentleman': 2, 'Sheriff': 2, 'Mariner': 3, 'Blockmaker': 2, 'as treasurer of the State of south carolina': 2, 'Clerk of the Senate': 2, 'Bricklayer': 1, 'Penniston': 1, 'as Guardian to Mary Deborah L. Gowdey': 1, 'Shipwright': 1, 'Cabinet Maker': 1, 'Autioneer': 1, 'Printer': 1, 'Assignee of James Simons': 1, 'Trustee for the Cincinnati of South Carolina': 1, 'Executor Philip Hawkins': 1, 'Vendue Master': 1, 'Executor of William Parker': 8, 'Hair Dresser': 1, 'Executrix of William Savage': 1, 'Comtroller of the Customs': 1, 'Tanner': 1, 'Blacks smith': 1, 'One of Judges of the Courts': 1, 'Planter as Executor Ely Kershaw': 1, 'Executor to Richard Mason': 1, 'Merchant  as Executor to Benjamin Garden': 1, 'Spinster': 1, 'Administrator to Joseph Warley': 2}


In [75]:
exceptions = ['Blacks smith', 'Cabinet Maker', 'Hair Dresser', 'One of Judges of the Courts', 'Vendue Master', 
              'as Guardian to Mary Deborah L. Gowdey', 'as treasurer of the State of south carolina'] #applicable only to a few states

In [76]:
#get the first word from every occupation and store any duplicates in a dictionary
occ_list = occs_debt_wo_dups['Occupation'].to_list()
occupations = []
one_word_occs = {}
flipped_one_w = {}

for occ in occ_list:
    if occ not in exceptions:
        first_word = occ.split()[0]
        one_word_occs[occ_list.index(occ)] = first_word 
        
#flip dictionary
for key, value in one_word_occs.items():
    flipped_one_w.setdefault(value, list()).append(key) #flip keys with values

In [77]:
#use fuzzy string matching 
flipped_list = list(flipped_one_w) #convert to list. needed in order to iterate through
for occ in flipped_list:
    for occ_2 in flipped_list:
        if occ_2 != occ and rapidfuzz.fuzz.ratio(occ, occ_2) >= 80:
            flipped_one_w[occ].extend(flipped_one_w[occ_2]) #combine lists
            flipped_one_w.pop(occ_2) #remove second occurence of the same occupation 
            flipped_list.remove(occ_2) #remove from list 
            
keys_w_dups = [key for key, values in flipped_one_w.items() if len(values) > 1] #retrieve keys that have multiple values. store them in a list
         
print(flipped_one_w)
print("\n\n")
print(keys_w_dups)

{'Administrator': [0], 'Assignee': [1], 'Attorney': [2], 'Autioneer': [3], 'Blockmaker': [5], 'Bricklayer': [6], 'Broker': [7], 'Carpenter': [9], 'Clerk': [10], 'Comtroller': [11], 'Executor': [12, 13, 14, 15], 'Factor': [16], 'Gentleman': [17], 'Mariner': [19], 'Merchant': [20, 21], 'Penniston': [23], 'Physician': [24], 'Planter': [25, 26], 'Printer': [27], 'Sheriff': [28], 'Shipwright': [29], 'Shopkeeper': [30], 'Spinster': [31], 'Tanner': [32], 'Trustee': [33], 'Widow': [35]}



['Executor', 'Merchant', 'Planter']


In [78]:
#remove duplicates
for occ in keys_w_dups:
    i = 0
    while i < len(flipped_one_w[occ]):
        if len(flipped_one_w[occ]) > 1:
            
            index = flipped_one_w[occ][i]
            last_index = flipped_one_w[occ][len(flipped_one_w[occ]) - 1] #get last value in list. we will add dollars and cents here
            duplicate_row = occs_debt_wo_dups.loc[[index]] #returns row that has index
            dollars_in_dup_row = duplicate_row[['Dollars', 'Dollars.1', 'Dollars.2']].sum(axis=1).values[0]
            cents_in_dup_row = duplicate_row[['Cents', 'Cents.1', 'Cents.2']].sum(axis=1).values[0]
            
            occs_debt_wo_dups.loc[last_index, 'Dollars'] += dollars_in_dup_row
            occs_debt_wo_dups.loc[last_index, 'Cents'] += cents_dup_row
            
            occurences_dict[occs_debt_wo_dups.loc[last_index, 'Occupation']] += occurences_dict[occs_debt_wo_dups.loc[index, 'Occupation']] 
            occurences_dict.pop(occs_debt_wo_dups.loc[index, 'Occupation'])

            occs_debt_wo_dups.drop(index, inplace=True)
            flipped_one_w[occ].remove(index)
            
        else:
            i += 1

occs_debt_wo_dups = occs_debt_wo_dups.reset_index(drop=True) #reset indexes 0-...

print(occs_debt_wo_dups.to_string())

                                      Occupation   Dollars  Dollars.1  Dollars.2       Cents     Cents.1  Cents.2
0                 Administrator to Joseph Warley    1750.0      835.0      601.0  211.000000   72.000000     64.0
1                       Assignee of James Simons     345.0      172.0      321.0    2.000000   51.000000     84.0
2                                Attorney at Law    3688.0     1842.0     2286.0  294.000000  351.000000    365.0
3                                      Autioneer      10.0        5.0        0.0   65.000000   32.000000      0.0
4                                   Blacks smith    1423.0      711.0      365.0   48.000000   74.000000     55.0
5                                     Blockmaker    5263.0     2631.0     2697.0  123.000000  111.000000     60.0
6                                     Bricklayer      37.0       18.0       18.0   35.000000   68.000000     75.0
7                                         Broker    2330.0     1162.0      929.0  489.00

In [79]:
occs_debt_wo_dups[['Cents', 'Cents.1', 'Cents.2']] = occs_debt_wo_dups[['Cents', 'Cents.1', 'Cents.2']].div(100)
print(occs_debt_wo_dups.to_string())

                                      Occupation   Dollars  Dollars.1  Dollars.2     Cents   Cents.1  Cents.2
0                 Administrator to Joseph Warley    1750.0      835.0      601.0  2.110000  0.720000     0.64
1                       Assignee of James Simons     345.0      172.0      321.0  0.020000  0.510000     0.84
2                                Attorney at Law    3688.0     1842.0     2286.0  2.940000  3.510000     3.65
3                                      Autioneer      10.0        5.0        0.0  0.650000  0.320000     0.00
4                                   Blacks smith    1423.0      711.0      365.0  0.480000  0.740000     0.55
5                                     Blockmaker    5263.0     2631.0     2697.0  1.230000  1.110000     0.60
6                                     Bricklayer      37.0       18.0       18.0  0.350000  0.680000     0.75
7                                         Broker    2330.0     1162.0      929.0  4.890000  5.470000     3.04
8         

In [80]:
occs_debt_wo_dups['Total'] = occs_debt_wo_dups[['Dollars', 'Dollars.1', 'Dollars.2', 'Cents', 'Cents.1', 'Cents.2']].sum(axis=1) #add totals to new column
print(occs_debt_wo_dups.to_string())

                                      Occupation   Dollars  Dollars.1  Dollars.2     Cents   Cents.1  Cents.2      Total
0                 Administrator to Joseph Warley    1750.0      835.0      601.0  2.110000  0.720000     0.64    3189.47
1                       Assignee of James Simons     345.0      172.0      321.0  0.020000  0.510000     0.84     839.37
2                                Attorney at Law    3688.0     1842.0     2286.0  2.940000  3.510000     3.65    7826.10
3                                      Autioneer      10.0        5.0        0.0  0.650000  0.320000     0.00      15.97
4                                   Blacks smith    1423.0      711.0      365.0  0.480000  0.740000     0.55    2500.77
5                                     Blockmaker    5263.0     2631.0     2697.0  1.230000  1.110000     0.60   10593.94
6                                     Bricklayer      37.0       18.0       18.0  0.350000  0.680000     0.75      74.78
7                               

In [81]:
occs_debt_wo_dups['# of People'] = occs_debt_wo_dups['Occupation'].map(occurences_dict) #add occurences to table
print(occs_debt_wo_dups[['Occupation', '# of People']])

                                      Occupation  # of People
0                 Administrator to Joseph Warley            2
1                       Assignee of James Simons            1
2                                Attorney at Law            8
3                                      Autioneer            1
4                                   Blacks smith            1
5                                     Blockmaker            2
6                                     Bricklayer            1
7                                         Broker           11
8                                  Cabinet Maker            1
9                                      Carpenter            5
10                           Clerk of the Senate            2
11                     Comtroller of the Customs            1
12                   Executrix of William Savage           11
13                                        Factor            3
14                                     Gentleman            2
15      

In [82]:
occs_debt_wo_dups['Average'] = occs_debt_wo_dups['Total'] / occs_debt_wo_dups['# of People'] #calculate average and add to new column
print(occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average']].to_string())

                                      Occupation      Total  # of People      Average
0                 Administrator to Joseph Warley    3189.47            2  1594.735000
1                       Assignee of James Simons     839.37            1   839.370000
2                                Attorney at Law    7826.10            8   978.262500
3                                      Autioneer      15.97            1    15.970000
4                                   Blacks smith    2500.77            1  2500.770000
5                                     Blockmaker   10593.94            2  5296.970000
6                                     Bricklayer      74.78            1    74.780000
7                                         Broker    4434.40           11   403.127273
8                                  Cabinet Maker     111.69            1   111.690000
9                                      Carpenter     686.92            5   137.384000
10                           Clerk of the Senate    26

In [83]:
exceptions = ['Blacks smith', 'Cabinet Maker', 'Hair Dresser', 'One of Judges of the Courts', 'Vendue Master', 
              'as Guardian to Mary Deborah L. Gowdey', 'as treasurer of the State of south carolina'] #applicable only to a few states

In [84]:
#convert to one word
def first_word_only(occ):
    if 'as Guardian' in occ:
        return 'Guardian'
    if 'as treasurer' in occ:
        return 'State Treasurer'
    if 'Blacks' in occ:
        return 'Blacksmith'
    if occ not in exceptions:
        return occ.split()[0]
    else:
        return occ
    
occs_debt_wo_dups['Occupation'] = occs_debt_wo_dups['Occupation'].apply(first_word_only)
print(occs_debt_wo_dups.to_string())

                     Occupation   Dollars  Dollars.1  Dollars.2     Cents   Cents.1  Cents.2      Total  # of People      Average
0                 Administrator    1750.0      835.0      601.0  2.110000  0.720000     0.64    3189.47            2  1594.735000
1                      Assignee     345.0      172.0      321.0  0.020000  0.510000     0.84     839.37            1   839.370000
2                      Attorney    3688.0     1842.0     2286.0  2.940000  3.510000     3.65    7826.10            8   978.262500
3                     Autioneer      10.0        5.0        0.0  0.650000  0.320000     0.00      15.97            1    15.970000
4                    Blacksmith    1423.0      711.0      365.0  0.480000  0.740000     0.55    2500.77            1  2500.770000
5                    Blockmaker    5263.0     2631.0     2697.0  1.230000  1.110000     0.60   10593.94            2  5296.970000
6                    Bricklayer      37.0       18.0       18.0  0.350000  0.680000     0.

In [85]:
total = occs_debt_wo_dups['Total'].sum()
occs_debt_wo_dups['% of Total Debt'] = round((occs_debt_wo_dups['Total'] / total) * 100, 3)
print(occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average', '% of Total Debt']].to_string())

                     Occupation      Total  # of People      Average  % of Total Debt
0                 Administrator    3189.47            2  1594.735000            1.089
1                      Assignee     839.37            1   839.370000            0.286
2                      Attorney    7826.10            8   978.262500            2.671
3                     Autioneer      15.97            1    15.970000            0.005
4                    Blacksmith    2500.77            1  2500.770000            0.853
5                    Blockmaker   10593.94            2  5296.970000            3.615
6                    Bricklayer      74.78            1    74.780000            0.026
7                        Broker    4434.40           11   403.127273            1.513
8                 Cabinet Maker     111.69            1   111.690000            0.038
9                     Carpenter     686.92            5   137.384000            0.234
10                        Clerk    2618.73            

In [86]:
#final total debt held by people w/ no job listed
occs_and_debt = pd.read_excel(path,
                             header=11, usecols='G, H, M, N, AB, AC, AQ, AR') #occupations plus continental debt for conneticut
no_occ = occs_and_debt[occs_and_debt['Occupation'].isna()]
num_of_people = len(no_occ)
no_occ = no_occ.reset_index(drop=True)
no_occ[['Cents', 'Cents.1', 'Cents.2']] = no_occ[['Cents', 'Cents.1', 'Cents.2']].div(100)
total_series = no_occ[['Dollars', 'Dollars.1', 'Dollars.2', 'Cents', 'Cents.1', 'Cents.2']].sum(axis=1)
total = total_series.sum()

occs_debt_wo_dups = occs_debt_wo_dups.append({'Occupation': 'None Listed', 'Total': total, '# of People': num_of_people, 'Average': total / num_of_people}, ignore_index=True)
print(occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average', '% of Total Debt']].to_string())

                     Occupation       Total  # of People       Average  % of Total Debt
0                 Administrator     3189.47            2   1594.735000            1.089
1                      Assignee      839.37            1    839.370000            0.286
2                      Attorney     7826.10            8    978.262500            2.671
3                     Autioneer       15.97            1     15.970000            0.005
4                    Blacksmith     2500.77            1   2500.770000            0.853
5                    Blockmaker    10593.94            2   5296.970000            3.615
6                    Bricklayer       74.78            1     74.780000            0.026
7                        Broker     4434.40           11    403.127273            1.513
8                 Cabinet Maker      111.69            1    111.690000            0.038
9                     Carpenter      686.92            5    137.384000            0.234
10                        Clerk 

In [87]:
occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average', '% of Total Debt']].to_excel("./FinishedSpreadsheets/occupation_debt_" + state + ".xlsx")